### En este notebook hago una prueba de implementación del algoritmo de market basket 'a priori' con el objetivo de implementarlo en casos más específicos en un futuro.

##### También aprovecho para sacar días y horas de mayores ventas

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

In [ ]:
ventas = pd.read_csv(r'C:\Users\pablo\OneDrive - Helacor S.A\Club Grido\Extractos DW\ventas_primer_semestre_21-22.csv')
ventas.head()

In [ ]:
pd.DataFrame(ventas.ProductName.value_counts(normalize=True)*100).head(10)

##### Analizo las horas de venta

In [ ]:
horas = pd.to_datetime(ventas['SaleDate']).dt.hour

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.tick_params(axis='x', labelrotation = 30)
plt.title('Compras por hora del día')
fig = sns.countplot(y=horas, palette=["#0C3B65"])
ax.set_xlabel('Cantidad de compras',loc='center')
ax.set_ylabel('Hora del día en que se compra',loc='center')

##### Analizo los días de venta

In [ ]:
import locale
import calendar
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')
dia = pd.to_datetime(ventas['SaleDate']).dt.weekday.apply(lambda x: calendar.day_name[x])

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.tick_params(axis='x', labelrotation = 30)
plt.title('Compras por día de la semana')
fig = sns.countplot(y=dia, order = list(calendar.day_name), color='grey')
ax.set_xlabel('Cantidad de compras',loc='center')
ax.set_ylabel('',loc='center')

In [ ]:
(dia.value_counts(normalize=True)*100).apply(lambda x: round(x,1))

##### Proceso data frame

In [ ]:
ventas = ventas[pd.to_datetime(ventas.SaleDate).dt.date >= dt.datetime(2021,8,1).date()]

In [ ]:
ventas=ventas[['SaleId','Amount','ProductId','ProductName']]

In [ ]:
productos = pd.read_excel(r'C:\Users\Pablo\Helacor S.A\Equipo CLUB GRIDO - General\Datos\productos_vigentes.xlsx')
productos['ProductId']= productos.ProductId.astype('float')
productos=productos[productos['Linea x Producto']!='No Definido']
productos.drop_duplicates(subset=['ProductId'],inplace=True)
productos.drop('ProductName',axis=1,inplace=True)
productos.head()

In [ ]:
ventas.shape

In [ ]:
ventas= ventas.merge(productos,left_on='ProductId',right_on='ProductId',how='left')
ventas.shape

In [ ]:
ventas.head()

In [ ]:
ventas.isna().sum()

In [ ]:
ventas.dropna(inplace=True)

In [ ]:
ventas.ProductName.value_counts(normalize=True)*100

In [ ]:
ventas.familia.value_counts(normalize=True)*100

In [ ]:
ventas[ventas['Linea x Producto']=='Líneas Especiales'].head()

#### Armo la columna "Linea" donde estará linea o familia de producto, segun volumen

In [ ]:
ventas['Linea x Producto'].value_counts(normalize=True)*100

In [ ]:
ventas['Linea'] = ventas['Linea x Producto']

ventas['Linea'] = np.where((ventas['familia']=='Frizzio') | (ventas['Linea']=='Frizzio'),'Congelado',ventas['Linea'])
ventas['Linea'] = np.where((ventas['familia']=='Gofriz')|(ventas['familia']=='McCain') | (ventas['familia']=='Swift'),'Congelado',ventas['Linea'])
ventas['Linea'] = np.where((ventas['Linea']=='Cañon')|(ventas['Linea']=='Conosud')|(ventas['Linea']=='Black Pan')\
                           |(ventas['Linea']=='Granja del Sol')|(ventas['Linea']=='Cresfood')|(ventas['Linea']=='Bettini'),
                          'Congelado',ventas['Linea'])

ventas['Linea'] = np.where(ventas['Linea']=='Helado x kg','Helado X Kilo',ventas['Linea'])
ventas['Linea'] = np.where(ventas['Linea']=='Factura','Café',ventas['Linea'])
ventas['Linea'] = np.where(ventas['Linea']=='Especial','Líneas Especiales',ventas['Linea'])
ventas['Linea'] = np.where((ventas['Linea']=='Cups y vasos')|(ventas['Linea']=='Tops'),'A definir',ventas['Linea'])

In [ ]:
ventas['Linea'].value_counts(normalize=True)*100

Armo el df para correr el algoritmo

In [ ]:
basket = ventas.groupby(['SaleId', 'Linea'])['Amount'].sum().unstack().reset_index().fillna(0).set_index('SaleId')
basket.dtypes

In [ ]:
basket.head()

In [ ]:
basket.columns

In [1]:
# Matriz esparsa

def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

In [ ]:
basket_encoded = basket.applymap(hot_encode)
basket = basket_encoded
basket= basket.astype('int32')

In [ ]:
# Building the model
df= basket[['Bombones', 'Congelado', 'Familiar','Helado X Kilo', 'Helado x Bocha', 'Palitos','Postres', 'Pote 1 Litro', 'Semifrío', 'Tortas']]
#df = df.loc[~(df==0).all(axis=1)]
frq_items = apriori(df, min_support = 0.01, use_colnames = True)


# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules

In [ ]:
frq_items

In [ ]:
rules